In [1]:
# Условие: 
# используйте источник rate, напишите код, который создаст дополнительный столбец, 
# который будет выводить сумму только нечётных чисел.

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, monotonically_increasing_id, collect_list, lit, aggregate

ImportError: cannot import name 'aggregate' from 'pyspark.sql.functions' (/usr/local/lib/python3.7/dist-packages/pyspark/sql/functions.py)

In [3]:
spark = SparkSession.builder.appName('Seminar#4-hw').getOrCreate()

24/04/26 17:07:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
streaming_df = spark.readStream.format('rate').option("rowsPerSecond", 1).load()

In [8]:
# Покажем, что действительно выбираются только нечетные числа в отдельный столбец, при этом отдельные отсчеты не пропускаются.
df_sum_odd = streaming_df.selectExpr('max(timestamp) as timestamp', 'max(value) as value','filter(collect_list(value), x -> x % 2 > 0) as odd_values_list')
query = df_sum_odd.writeStream.outputMode('update').format('console').start()
query.awaitTermination(10)
query.stop()

24/04/26 17:10:59 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-769bda49-5ffc-4077-9355-10513dba4469. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+---------------+
|timestamp|value|odd_values_list|
+---------+-----+---------------+
|     null| null|             []|
+---------+-----+---------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+---------------+
|           timestamp|value|odd_values_list|
+--------------------+-----+---------------+
|2024-04-26 17:10:...|    0|             []|
+--------------------+-----+---------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+---------------+
|           timestamp|value|odd_values_list|
+--------------------+-----+---------------+
|2024-04-26 17:11:...|    1|            [1]|
+--------------------+-----+---------------+

-------------------------------------------
Batch: 3
---------------------------------

In [42]:
#  Вывод только суммы нечетных чисел
df_sum_odd = \
    streaming_df.selectExpr('max(timestamp) as timestamp', 'max(value) as value','filter(collect_list(value), x -> x % 2 > 0) as odd_values').\
    selectExpr('*','aggregate(odd_values, bigint(0), (x, y) -> (x) + (y)) as sum_odd')
query = df_sum_odd.writeStream.outputMode('update').format('console').start()
query.awaitTermination(10)
query.stop()

24/04/26 19:14:09 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-54af5d24-2327-4d1f-8306-cf60d8aa7209. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+----------+-------+
|timestamp|value|odd_values|sum_odd|
+---------+-----+----------+-------+
|     null| null|        []|      0|
+---------+-----+----------+-------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+----------+-------+
|           timestamp|value|odd_values|sum_odd|
+--------------------+-----+----------+-------+
|2024-04-26 19:14:...|    0|        []|      0|
+--------------------+-----+----------+-------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+----------+-------+
|           timestamp|value|odd_values|sum_odd|
+--------------------+-----+----------+-------+
|2024-04-26 19:14:...|    1|       [1]|      1|
+--------------------+-----+----------+-------+

-----------------------------------------